In [42]:
import statsapi
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import pandas as pd

In [6]:
yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

In [7]:
games = statsapi.schedule(start_date=yesterday, end_date=yesterday)

In [9]:
game_id = 778022

In [8]:
games

[{'game_id': 778022,
  'game_datetime': '2025-05-07T16:05:00Z',
  'game_date': '2025-05-07',
  'game_type': 'R',
  'status': 'Final',
  'away_name': 'Cleveland Guardians',
  'home_name': 'Washington Nationals',
  'away_id': 114,
  'home_id': 120,
  'doubleheader': 'N',
  'game_num': 1,
  'home_probable_pitcher': 'Michael Soroka',
  'away_probable_pitcher': 'Logan Allen',
  'home_pitcher_note': '',
  'away_pitcher_note': '',
  'away_score': 8,
  'home_score': 6,
  'current_inning': 9,
  'inning_state': 'Bottom',
  'venue_id': 3309,
  'venue_name': 'Nationals Park',
  'national_broadcasts': [],
  'series_status': 'CLE wins 2-1',
  'winning_team': 'Cleveland Guardians',
  'losing_team': 'Washington Nationals',
  'winning_pitcher': 'Joey Cantillo',
  'losing_pitcher': 'Michael Soroka',
  'save_pitcher': 'Emmanuel Clase',
  'summary': '2025-05-07 - Cleveland Guardians (8) @ Washington Nationals (6) (Final)'},
 {'game_id': 778015,
  'game_datetime': '2025-05-07T17:10:00Z',
  'game_date': '20

In [18]:
print(statsapi.boxscore(game_id))

------------------------------------------------------------------------------- | -------------------------------------------------------------------------------
Guardians Batters                        AB   R   H  RBI BB   K  LOB AVG   OPS  | Nationals Batters                        AB   R   H  RBI BB   K  LOB AVG   OPS 
------------------------------------------------------------------------------- | -------------------------------------------------------------------------------
1 Kwan  DH                                3   2   1   0   2   0   0  .333 .854  | 1 Abrams  SS                              6   1   3   1   0   0   2  .286 .855 
2 Schneemann  LF                          4   1   1   0   1   2   1  .286 .938  | 2 Rosario, A  3B                          5   1   3   2   1   1   2  .279 .741 
3 Ramírez, Jo  3B                         4   1   1   1   0   2   0  .271 .775  | 3 Wood  DH                                6   0   3   0   0   2   3  .281 .937 
4 Santana  1B               

In [21]:
print(statsapi.linescore(game_id))

Final     1 2 3 4 5 6 7 8 9  R   H   E  
Guardians 0 0 0 0 0 8 0 0 0  8   10  1  
Nationals 0 1 2 0 0 0 3 0 0  6   13  0  


In [22]:
line = statsapi.linescore(game_id)
lines = line.strip().split('\n')

In [23]:
lines

['Final     1 2 3 4 5 6 7 8 9  R   H   E  ',
 'Guardians 0 0 0 0 0 8 0 0 0  8   10  1  ',
 'Nationals 0 1 2 0 0 0 3 0 0  6   13  0']

In [ ]:
inning_numbers = lines[0].split()[1:-3]  # '1 2 3 ... 9'
team1_scores = lines[1].split()[1:-3]    # Guardians
team2_scores = lines[2].split()[1:-3]

In [29]:
target_inning = 2
team1_score_2nd = int(team1_scores[target_inning-1])
team2_score_2nd = int(team2_scores[target_inning-1])

In [31]:
team2_score_2nd

1

### Fetching games for week

In [74]:
end_date = datetime.now() - timedelta(days=1)
start_date = end_date - timedelta(days=30)
start_str = start_date.strftime('%Y-%m-%d')
end_str = end_date.strftime('%Y-%m-%d')

In [75]:
games = statsapi.schedule(start_date=start_str, end_date=end_str)

In [76]:
data = []

for game in tqdm(games):
    try:
        game_id = game['game_id']
        home = game['home_name']
        away = game['away_name']
        date = game['game_date']

        # Финальный счёт
        final_score = f"{game['home_score']} - {game['away_score']}"

        # Линия иннингов
        linescore_text = statsapi.linescore(game_id)
        lines = linescore_text.strip().split('\n')

        if len(lines) >= 3:
            away_scores = lines[1].split()[1:-3]
            home_scores = lines[2].split()[1:-3]
            if len(away_scores) >= 2 and len(home_scores) >= 2:
                score_2nd = f"{home_scores[1]} - {away_scores[1]}"
            else:
                score_2nd = "N/A"
        else:
            score_2nd = "N/A"

        data.append({
            "home": home,
            "away": away,
            "score": final_score,
            "score_2_inning": score_2nd,
            "date": date,
            "home_score": int(game['home_score']),
            "away_score": int(game['away_score']),
            "home_score_2nd": int(home_scores[1]),
            "away_score_2nd": int(away_scores[1]),
        })

    except Exception:
        continue


  0%|          | 0/420 [00:00<?, ?it/s]

In [77]:
df = pd.DataFrame(data)

In [78]:
df.head()

,home,away,score,score_2_inning,date,home_score,away_score,home_score_2nd,away_score_2nd
0,Detroit Tigers,New York Yankees,6 - 2,0 - 0,2025-04-07,6,2,0,0
1,Pittsburgh Pirates,St. Louis Cardinals,8 - 4,3 - 0,2025-04-07,8,4,3,0
2,Washington Nationals,Los Angeles Dodgers,6 - 4,2 - 0,2025-04-07,6,4,2,0
3,Boston Red Sox,Toronto Blue Jays,2 - 6,0 - 0,2025-04-07,2,6,0,0
4,New York Mets,Miami Marlins,2 - 0,0 - 0,2025-04-07,2,0,0,0


In [79]:
df['games_played'] = 1
df['second_inning_scored'] = ((df['home_score_2nd'] > 0) | (df['away_score_2nd'] > 0)).astype(int)

In [ ]:
summary = df.groupby('date').agg(
    second_inning_scored_count=('second_inning_scored', 'sum'),
    total_games=('games_played', 'sum')
).reset_index()

summary.columns = ['date', 'second_inning_scored_count', 'total_games']

In [90]:
summary["revenue_total_under"] = 1.65 * (summary["total_games"] - summary["second_inning_scored_count"]) - summary["total_games"]
summary["revenue_total_over"] = 1.9 * (summary["second_inning_scored_count"]) - summary["total_games"]

In [91]:
summary

,date,second_inning_scored_count,total_games,revenue_total_under,revenue_total_over
0,2025-04-07,4,11,0.55,-3.4
1,2025-04-08,6,15,-0.15,-3.6
2,2025-04-09,5,15,1.50,-5.5
3,2025-04-10,3,6,-1.05,-0.3
4,2025-04-11,5,14,0.85,-4.5
5,2025-04-12,5,15,1.50,-5.5
6,2025-04-13,8,15,-3.45,0.2
7,2025-04-14,3,10,1.55,-4.3
8,2025-04-15,7,15,-1.80,-1.7
9,2025-04-16,5,15,1.50,-5.5


In [92]:
summary[summary["revenue_total_under"] > 0].shape

(13, 5)

In [94]:
summary[summary["revenue_total_over"] > 0].shape

(11, 5)